In [1]:
import numpy as np
from dataclasses import dataclass
import math
from typing import Literal, List
import gsw  # TEOS-10 library (for salinity)

In [2]:
DBAR_TO_PSI = 1.450377
PSI_TO_DBAR = 0.6894759
OXYGEN_PHASE_TO_VOLTS = 39.457071
KELVIN_OFFSET_0C = 273.15
KELVIN_OFFSET_25C = 298.15
OXYGEN_MLPERL_TO_MGPERL = 1.42903
OXYGEN_MLPERL_TO_UMOLPERKG = 44.660
# taken from https://blog.seabird.com/ufaqs/what-is-the-difference-in-temperature-expressions-between-ipts-68-and-its-90/ # pylint: disable=line-too-long
ITS90_TO_IPTS68 = 1.00024
# micro moles of nitrate to milligrams of nitrogen per liter
UMNO3_TO_MGNL = 0.014007
# [J K^{-1} mol^{-1}] Gas constant, NIST Reference on Constants retrieved 10-05-2015
R = 8.3144621
# [Coulombs mol^{-1}] Faraday constant, NIST Reference on Constants retrieved 10-05-2015
F = 96485.365

In [3]:
class TemperatureCoefficients:
    """
    :param a0: coefficient
    :param a1: coefficient
    :param a2: coefficient
    :param a3: coefficient
    """

    a0: float
    a1: float
    a2: float
    a3: float

class PressureCoefficients:
    C1: float
    C2: float
    C3: float
    D1: float
    D2: float
    T1: float
    T2: float
    T3: float
    T4: float
    T5: float
    Slope: float
    Offset: float
    AD590M: float
    AD590B: float

class ConductivityCoefficients:
    """
    :param g: coefficient
    :param h: coefficient
    :param i: coefficient
    :param j: coefficient
    :param cpcor: compressibility coefficient
    :param ctcor: coefficient
    :param wbotc: bridge oscillator temperature coefficient see the
        37 Manual: https://www.seabird.com/asset-get.download.jsa?id=54627862348
    """

    g: float
    h: float
    i: float
    j: float
    cpcor: float
    ctcor: float
    wbotc: float

class Oxygen43Coefficients:
    """
    :param soc: linear scaling coefficient
    :param v_offset: voltage at zero oxygen signal
    :param tau_20: sensor time constant tau(T,P) at 20 C, 1 atmosphere, 0 PSU;
        slope term in calculation of tau(T,P)
    :param a: coefficient
    :param b: coefficient
    :param c: coefficient
    :param e: coefficient
    :param d0: tau(T,P) coefficient
    :param d1: tau(T,P) coefficient
    :param d2: tau(T,P) coefficient
    :param h1: hysteresis correction coefficient
    :param h2: hysteresis correction coefficient
    :param h3: hysteresis correction coefficient
    """

    soc: float
    v_offset: float
    tau_20: float
    a: float
    b: float
    c: float
    e: float
    d0: float  # TODO: the sensor lists a d0 coefficient, but it doesn't seem to be used?
    d1: float
    d2: float
    h1: float
    h2: float
    h3: float

In [4]:
class PARCoefficients:
    """
    :param im: immersion coefficient
    :param a0: calibration slope
    :param a1: calibration offset
    :param multiplier: 1.0 for units of μEinsteins/m2 sec
    """

    im: float
    a0: float
    a1: float
    multiplier: float

class PAR_Coefficients:
    m: float
    b: float
    calicon: float #CalibrationConstant
    multiplier: float
    offset: float

class Altimeter_Coefficients:
    scalefactor: float
    offset: float

class FluroWetlab_Coefficients:
    scalefactor: float
    Vblank: float

class Turbidity_Coefficients:
    scalefactor: float
    darkvoltage: float

class Spar_Coefficients:
    conversionfactor: float
    ratiomultiplier: float



In [5]:
temp1_coefs = TemperatureCoefficients()
temp1_coefs.a0 = 4.33459406e-003
temp1_coefs.a1 = 6.28735680e-004
temp1_coefs.a2 = 2.05782221e-005
temp1_coefs.a3 = 1.68624541e-006

temp2_coefs = TemperatureCoefficients()
temp2_coefs.a0 = 4.34471154e-003
temp2_coefs.a1 = 6.38146193e-004
temp2_coefs.a2 = 2.04525401e-005
temp2_coefs.a3 = 1.60292340e-006


pres_coefs = PressureCoefficients()
pres_coefs.C1 = -4.212464e+004
pres_coefs.C2 = -1.972709e-001
pres_coefs.C3 = 1.365930e-002

pres_coefs.D1 = 3.604100e-002
pres_coefs.D2 = 0.000000e+000

pres_coefs.T1 = 3.046128e+001
pres_coefs.T2 = -3.791930e-004
pres_coefs.T3 = 4.202260e-006
pres_coefs.T4 = 3.386190e-00
pres_coefs.T5 = 0.000000e+000

pres_coefs.Slope = 1.00000000
pres_coefs.Offset = -0.20000
pres_coefs.AD590M = 1.279410e-002
pres_coefs.AD590B = -9.215740e+000



cond1_coefs = ConductivityCoefficients()
cond1_coefs.g = -9.94900904e+000
cond1_coefs.h = 1.42164741e+000
cond1_coefs.i = 1.26008747e-004
cond1_coefs.j = 6.16820694e-005
cond1_coefs.cpcor = -9.57000000e-008
cond1_coefs.ctcor = 3.2500e-006
cond1_coefs.wbotc = 0.00000000e+000

cond2_coefs = ConductivityCoefficients()
cond2_coefs.g = -9.77824825e+000
cond2_coefs.h = 1.31877668e+000
cond2_coefs.i = 2.26556605e-004
cond2_coefs.j = 6.59127902e-005
cond2_coefs.cpcor = -9.57000000e-008
cond2_coefs.ctcor = 3.2500e-006
cond2_coefs.wbotc = 0.00000000e+000


oxy1_coefs = Oxygen43Coefficients()
oxy1_coefs.soc = 4.7612e-001
oxy1_coefs.v_offset = -0.4891
oxy1_coefs.tau_20 = 1.7400
oxy1_coefs.a = -3.2939e-003
oxy1_coefs.b = 1.4617e-004
oxy1_coefs.c = -2.1408e-006
oxy1_coefs.e = 3.6000e-002
oxy1_coefs.d0 = 2.5826e+000
oxy1_coefs.d1 = 1.92634e-004
oxy1_coefs.d2 = -4.64803e-002
oxy1_coefs.h1 = -3.3000e-002
oxy1_coefs.h2 = 5.0000e+003
oxy1_coefs.h3 = 1.4500e+003

oxy2_coefs = Oxygen43Coefficients()
oxy2_coefs.soc = 5.4771e-001
oxy2_coefs.v_offset = -0.5020
oxy2_coefs.tau_20 = 1.2900
oxy2_coefs.a = -3.9126e-003
oxy2_coefs.b = 1.5415e-004
oxy2_coefs.c = -2.2575e-006
oxy2_coefs.e = 3.6000e-002
oxy2_coefs.d0 = 2.5826e+000
oxy2_coefs.d1 = 1.92634e-004
oxy2_coefs.d2 = -4.64803e-002
oxy2_coefs.h1 = -3.3000e-002
oxy2_coefs.h2 = 5.0000e+003
oxy2_coefs.h3 = 1.4500e+003



par_coefs = PARCoefficients()
par_coefs.im = 1.00000000
par_coefs.a0 = 11273957159.00000000
par_coefs.a1 = -0.09022432
par_coefs.multiplier = 1.00000000

In [6]:
hexstrings = [
    "0CD08410767B80458F0CCA6F1107AE7D58A3987C41F0DEF2FFFFBE0000BB29512A089DB7004D638E5C91C465",
    "0CD0811076768045960CCA6F1107B27D48A3986C40F0DEF2FFFFBD0000BB29512A089DB7004D638F5C91C465",
    "0CD08710767B8045960CCA6D1107B17D48A3986C40F10EF0FFFFBE0000BB29512A089DB7004D63905D91C465",
    "0CD08710767A80458F0CCA6C1107AE7D58A3986C3FF16EE7FFFFBE0000BB29512A089DB7004D63915D91C465",
    "0CD08A10767680459E0CCA6F1107B27D58A3987C40F18EE3FFFFBE0000BB29512A089DB7004D63925D91C465",
    "0CD08410767A8045960CCA6C1107B17D58A3988C41F18EE3FFFFBE0000BB29512A089DB7004D63935D91C465",
    "0CD0811076798045960CCA6A1107B17D58A498AC41F18EE1FFFFBE0000BB29512A089DB7004D63945D91C465",
    "0CD08410767C80458F0CCA6A1107B07D48A398BC40F15ED4FFFFBE0000BB29512A089DB7004D63955D91C465",
    "0CD08710767A8045960CCA6F1107B17D48A498CC41F13ECEFFFFBE0000BB29512A089DB7004D63965D91C465",
    "0CD08110767980459D0CCA6F1107B07D48A398DC41F13ECEFFFFBD0000BB29512A089DB7004D63975D91C465"
]
bytes_list_array = [[hex_string[i:i+2] for i in range(0, len(hex_string), 2)] for hex_string in hexstrings]
pressure_test_array = np.array([1.149,1.197,1.197,1.149,1.253,1.197,1.197,1.149,1.197,1.246])

hex_string = "0CD08410767B80458F0CCA6F1107AE7D58A3987C41F0DEF2FFFFBE0000BB29512A089DB7004D638E5C91C465"
bytes_list = [hex_string[i:i+2] for i in range(0, len(hex_string), 2)]

In [7]:
def hex_to_voltage(hex_value: str):
    clean_hex = hex_value.strip().upper()
    
    # Hex to Decimal
    N = int(clean_hex, 16)

    # Convert to volt
    V = 5 * (1 - (N / 4095))

    return V

def Time_elapsed(hexstrings, freq=24):
    """
    Computes elapsed time for a list of hex strings.
    Assumption: Each string = 1 measurement, starting at 0 s.
    
    :param hexstrings: List of hex strings (each string is one scan).
    :param freq: Sampling frequency in Hz (default 24 Hz).
    :return: List of elapsed times (in seconds).
    """
    dt = 1 / freq  # time interval per measurement
    elapsed = [i * dt for i in range(len(hexstrings))]
    return elapsed

def convert_par(
    raw_par: np.ndarray,
    coefs: PARCoefficients,
):
    """Converts a raw voltage value for PAR to µmol photons/m2*s.

    All equation information comes from application note 96

    :param raw_par: raw output voltage from PAR sensor
    :param coefs: calibration coefficients for the PAR sensor

    :return: converted PAR in µmol photons/m2*s
    """
    
    par = coefs.multiplier * ((10**9 * 10**(raw_par/coefs.im))/coefs.a0) + coefs.a1

    return par


def pressure_from_frequency(freq_hz: np.ndarray, temp_freq_hz: np.ndarray, coefs: PressureCoefficients) -> np.ndarray:
    """
    Berechnet den Druck [dbar] aus Digiquartz-Frequenz und Temperaturfrequenz (arrays erlaubt).

    :param freq_hz: gemessene Druckfrequenz [Hz]
    :param temp_freq_hz: gemessene Temperaturfrequenz [Hz]
    :param coefs: PressureCoefficients
    :return: Druck in dbar
    """
    tau = 1.0e6 / freq_hz        # Periode Drucksensor [µs]
    tau_temp = 1.0e6 / temp_freq_hz  # Periode Temperatursensor [µs]
    U = tau_temp                 # Temperaturkorrekturgröße

    # Paroscientific Gleichung
    C = coefs.C1 + coefs.C2 * U + coefs.C3 * U**2
    D = coefs.D1 + coefs.D2 * U
    tau0 = coefs.T1 + coefs.T2*U + coefs.T3*U**2 + coefs.T4*U**3 + coefs.T5*U**4

    x = 1.0 - (tau0**2 / tau**2)
    P = C * x * (1.0 - D * x)

    # Lineare Korrektur
    P = coefs.Slope * P + coefs.Offset
    return P

In [8]:
def convert_temperature(
    temperature_counts_in: np.ndarray,
    coefs: TemperatureCoefficients,
    standard: Literal["ITS90", "IPTS68"] = "ITS90",
    units: Literal["C", "F"] = "C",
    use_mv_r: bool = False,
):
    """Returns the value after converting it to degrees C, ITS-90.

    Data is expected to be raw data from an instrument in A/D counts

    :param temperature_counts_in: temperature value to convert in A/D
        counts
    :param coefs: calibration coefficients for the temperature sensor
    :param standard: whether to use ITS90 or to use IPTS-68 calibration
        standard
    :param units: whether to use celsius or to convert to fahrenheit
    :param use_mv_r: true to perform extra conversion steps required by
        some instruments (check the cal sheet to see if this is required)

    :return: temperature val converted to ITS-90 degrees C
    """

    if use_mv_r:
        mv = (temperature_counts_in - 524288) / 1.6e007
        r = (mv * 2.900e009 + 1.024e008) / (2.048e004 - mv * 2.0e005)
        temperature_counts = r
    else:
        temperature_counts = temperature_counts_in

    log_t = np.log(temperature_counts)
    temperature = (
        1 / (coefs.a0 + coefs.a1 * log_t + coefs.a2 * log_t**2 + coefs.a3 * log_t**3)
    ) - KELVIN_OFFSET_0C

    if standard == "IPTS68":
        temperature *= ITS90_TO_IPTS68
    if units == "F":
        temperature = temperature * 9 / 5 + 32  # Convert C to F

    return temperature



def convert_pressure(
    pressure_count: np.ndarray,
    compensation_voltage: np.ndarray,
    coefs: PressureCoefficients,
    units: Literal["dbar", "psia"] = "psia",
):
    """Converts pressure counts to PSIA (pounds per square inch, abolute).

    pressure_count and compensation_voltage are expected to be raw data
    from an instrument in A/D counts

    :param pressure_count: pressure value to convert, in A/D counts
    :param compensation_voltage: pressure temperature compensation
        voltage, in counts or volts depending on the instrument
    :param coefs: calibration coefficients for the pressure sensor
    :param units: whether or not to use psia or dbar as the returned
        unit type

    :return: pressure val in PSIA
    """
    sea_level_pressure = 14.7

    t = (
        coefs.ptempa0
        + coefs.ptempa1 * compensation_voltage
        + coefs.ptempa2 * compensation_voltage**2
    )
    x = pressure_count - coefs.ptca0 - coefs.ptca1 * t - coefs.ptca2 * t**2
    n = x * coefs.ptcb0 / (coefs.ptcb0 + coefs.ptcb1 * t + coefs.ptcb2 * t**2)
    pressure = coefs.pa0 + coefs.pa1 * n + coefs.pa2 * n**2 - sea_level_pressure

    if units == "dbar":
        pressure *= PSI_TO_DBAR

    return pressure


def convert_conductivity(
    conductivity_count: np.ndarray,
    temperature: np.ndarray,
    pressure: np.ndarray,
    coefs: ConductivityCoefficients,
):
    """Converts raw conductivity counts to S/m.

    Data is expected to be raw data from instrument in A/D counts

    :param conductivity_count: conductivity value to convert, in A/D
        counts
    :param temperature: reference temperature, in degrees C
    :param pressure: reference pressure, in dbar
    :param coefs: calibration coefficient for the conductivity sensor

    :return: conductivity val converted to S/m
    """
    f = conductivity_count * np.sqrt(1 + coefs.wbotc * temperature) / 1000
    numerator = coefs.g + coefs.h * f**2 + coefs.i * f**3 + coefs.j * f**4
    denominator = 1 + coefs.ctcor * temperature + coefs.cpcor * pressure
    return numerator / denominator


def convert_sbe43_oxygen(
    voltage: np.ndarray,
    temperature: np.ndarray,
    pressure: np.ndarray,
    salinity: np.ndarray,
    coefs: Oxygen43Coefficients,
    apply_tau_correction: bool = False,
    apply_hysteresis_correction: bool = False,
    window_size: float = 1,
    sample_interval: float = 1,
):
    """Returns the data after converting it to ml/l.

    voltage is expected to be in volts, temperature in deg c, pressure
    in dbar, and salinity in practical salinity (PSU). All equation
    information comes from the June 2013 revision of the SBE43 manual

    :param voltage: SBE43 voltage
    :param temperature: temperature value converted to deg C
    :param pressure: Converted pressure value from the attached CTD, in
        dbar
    :param salinity: Converted salinity value from the attached CTD, in
        practical salinity PSU
    :param coefs: calibration coefficients for the SBE43 sensor
    :param apply_tau_correction: whether or not to run tau correction
    :param apply_hysteresis_correction: whether or not to run hysteresis
        correction
    :param window_size: size of the window to use for tau correction, if
        applicable, in seconds
    :param sample_interval: sample rate of the data to be used for tau
        correction, if applicable. In seconds.

    :return: converted Oxygen values, in ml/l
    """
    # start with all 0 for the dvdt
    dvdt_values = np.zeros(len(voltage))
    if apply_tau_correction:
        # Calculates how many scans to have on either side of our median
        # point, accounting for going out of index bounds
        scans_per_side = floor(window_size / 2 / sample_interval)
        for i in range(scans_per_side, len(voltage) - scans_per_side):
            ox_subset = voltage[i - scans_per_side : i + scans_per_side + 1]

            time_subset = np.arange(
                0, len(ox_subset) * sample_interval, sample_interval, dtype=float
            )

            result = stats.linregress(time_subset, ox_subset)

            dvdt_values[i] = result.slope

    correct_ox_voltages = voltage.copy()
    if apply_hysteresis_correction:
        # Hysteresis starts at 1 because 0 can't be corrected
        for i in range(1, len(correct_ox_voltages)):
            # All Equation info from APPLICATION NOTE NO. 64-3
            d = 1 + coefs.h1 * (np.exp(pressure[i] / coefs.h2) - 1)
            c = np.exp(-1 * sample_interval / coefs.h3)
            ox_volts = correct_ox_voltages[i] + coefs.v_offset

            prev_ox_volts_new = correct_ox_voltages[i - 1] + coefs.v_offset
            ox_volts_new = ((ox_volts + prev_ox_volts_new * c * d) - (prev_ox_volts_new * c)) / d
            ox_volts_final = ox_volts_new - coefs.v_offset
            correct_ox_voltages[i] = ox_volts_final

    oxygen = _convert_sbe43_oxygen(
        correct_ox_voltages,
        temperature,
        pressure,
        salinity,
        coefs,
        dvdt_values,
    )
    return oxygen

def _convert_sbe43_oxygen(
    voltage: np.ndarray,
    temperature: np.ndarray,
    pressure: np.ndarray,
    salinity: np.ndarray,
    coefs: Oxygen43Coefficients,
    dvdt_value: np.ndarray,
):
    """Returns the data after converting it to ml/l.

    voltage is expected to be in volts, temperature in deg c, pressure
    in dbar, and salinity in practical salinity (PSU). All equation
    information comes from the June 2013 revision of the SBE43 manual.
    Expects that hysteresis correction is already performed on the
    incoming voltage, if desired.

    :param voltage: SBE43 voltage
    :param temperature: temperature value converted to deg C
    :param pressure: Converted pressure value from the attached CTD, in
        dbar
    :param salinity: Converted salinity value from the attached CTD, in
        practical salinity PSU
    :param coefs: calibration coefficients for the SBE43 sensor
    :param dvdt_value: derivative value of voltage with respect to time
        at this point. Expected to be 0 if not using Tau correction

    :return: converted Oxygen value, in ml/l
    """

    # Oxygen Solubility equation constants, From SBE43 Manual Appendix A
    a0 = 2.00907
    a1 = 3.22014
    a2 = 4.0501
    a3 = 4.94457
    a4 = -0.256847
    a5 = 3.88767
    b0 = -0.00624523
    b1 = -0.00737614
    b2 = -0.010341
    b3 = -0.00817083
    c0 = -0.000000488682

    ts = np.log((KELVIN_OFFSET_25C - temperature) / (KELVIN_OFFSET_0C + temperature))
    a_term = a0 + a1 * ts + a2 * ts**2 + a3 * ts**3 + a4 * ts**4 + a5 * ts**5
    b_term = salinity * (b0 + b1 * ts + b2 * ts**2 + b3 * ts**3)
    c_term = c0 * salinity**2
    solubility = np.exp(a_term + b_term + c_term)

    # Tau correction
    tau = coefs.tau_20 * np.exp(coefs.d1 * pressure + coefs.d2 * (temperature - 20)) * dvdt_value

    soc_term = coefs.soc * (voltage + coefs.v_offset + tau)
    temp_term = 1.0 + coefs.a * temperature + coefs.b * temperature**2 + coefs.c * temperature**3
    oxygen = (
        soc_term
        * solubility
        * temp_term
        * np.exp((coefs.e * pressure) / (temperature + KELVIN_OFFSET_0C))
    )
    return oxygen

def convert_oxygen_to_umol_per_kg(ox_values: np.ndarray, potential_density: np.ndarray):
    """Converts given oxygen values to milligrams/kg.

    Note: Sigma-Theta is expected to be calculated via gsw_sigma0,
    meaning is it technically potential density anomaly. Calculating
    using gsw_rho(SA, CT, p_ref = 0) results in actual potential
    density, but this function already does the converison, so values
    will need to have 1000 subtracted from them before being passed into
    this function. The function is done this way to stay matching to
    Application Note 64, but the results of either method are identical.

    :param ox_values: oxygen values, already converted to ml/L
    :param potential_density: potential density (sigma-theta) values.
        Expected to be the same length as ox_values

    :return: oxygen values converted to milligrams/Liter
    """

    oxygen_umolkg = (ox_values * OXYGEN_MLPERL_TO_UMOLPERKG) / (potential_density + 1000)
    return oxygen_umolkg

def potential_density_from_t_s_p(
    temperature: np.ndarray,
    salinity: np.ndarray,
    pressure: np.ndarray,
    lon=0.0,
    lat=0.0,
    reference_pressure=0.0,
):
    """Derive potential density from measured temperature, salinity, and
    pressure.

    :param temperature: Measure temperature, in degrees C
    :param salinity: Measured salinity, in practical salinity units
    :param pressure: Measured pressure, in decibars
    :param lon: Longitude
    :param lat: Latitude
    :param reference_pressure: Reference pressure in decibars. Defaults
        to 0.0.

    :return: Potential density in kg/m^3
    """

    absolute_salinity = gsw.SA_from_SP(salinity, pressure, lon, lat)
    conservative_temperature = gsw.CT_from_t(absolute_salinity, temperature, pressure)
    potential_density = (
        gsw.rho(absolute_salinity, conservative_temperature, reference_pressure) - 1000
    )
    return potential_density

def depth_from_pressure(
    pressure_in: np.ndarray,
    latitude: float,
    depth_units: Literal["m", "ft"] = "m",
    pressure_units: Literal["dbar", "psi"] = "dbar",
):
    """Derive depth from pressure and latitude.

    :param pressure: Numpy array of floats representing pressure, in
        dbar or psi
    :param latitude: Latitude (-90.0 to 90.0)
    :param depth_units: 'm' for meters, 'ft' for feet. Defaults to 'm'.
    :param pressure_units: 'dbar' for decibars, 'psi' for PSI. Defaults
        to 'dbar'.

    :return: A numpy array representing depth in meters or feet
    """
    pressure = pressure_in.copy()
    if pressure_units == "psi":
        pressure /= DBAR_TO_PSI

    depth = -gsw.z_from_p(pressure, latitude)

    if depth_units == "ft":
        depth *= 3.28084

    return depth


In [9]:
F0 = 1000
temperature_counts1 = []
temperature_counts2 = []

for byte_list in bytes_list_array:
    count1 = int(byte_list[0], 16) * 256 + int(byte_list[1], 16) + int(byte_list[2], 16) / 256
    count2 = int(byte_list[9], 16) * 256 + int(byte_list[10], 16) + int(byte_list[11], 16) / 256
    temperature_counts1.append(count1)
    temperature_counts2.append(count2)

temperature_counts1 = np.array(temperature_counts1)
temperature_counts2 = np.array(temperature_counts2)

temperature_counts1 = F0 / temperature_counts1
temperature_counts2 = F0 / temperature_counts2

temperatures1 = convert_temperature(temperature_counts1, coefs=temp1_coefs)
temperatures2 = convert_temperature(temperature_counts2, coefs=temp2_coefs)

print("Temperatures1:", temperatures1)
print("Temperatures2:", temperatures2)

Temperatures1: [3.56156397 3.56140342 3.56172452 3.56172452 3.56188507 3.56156397
 3.56140342 3.56156397 3.56172452 3.56140342]
Temperatures2: [3.56104427 3.56104427 3.56093531 3.56088084 3.56104427 3.56088084
 3.56077188 3.56077188 3.56104427 3.56104427]


In [10]:
'''
F0 = 1000
temperature_counts1 = []

for byte_list in bytes_list_array:
    count1 = int(byte_list[0], 16) * 256 + int(byte_list[1], 16) + int(byte_list[2], 16) / 256
    temperature_counts1.append(count1)

temperature_counts1 = np.array(temperature_counts1)


pressure_counts = []

for byte_list in bytes_list_array:
    count = int(byte_list[6], 16) * 256 + int(byte_list[7], 16) + int(byte_list[8], 16) / 256
    pressure_counts.append(count)

pressure_counts = np.array(pressure_counts)

pressure = pressure_from_frequency(pressure_counts, temperature_counts1, coefs=pres_coefs)
print("Druck in dbar:", pressure)
'''

'\nF0 = 1000\ntemperature_counts1 = []\n\nfor byte_list in bytes_list_array:\n    count1 = int(byte_list[0], 16) * 256 + int(byte_list[1], 16) + int(byte_list[2], 16) / 256\n    temperature_counts1.append(count1)\n\ntemperature_counts1 = np.array(temperature_counts1)\n\n\npressure_counts = []\n\nfor byte_list in bytes_list_array:\n    count = int(byte_list[6], 16) * 256 + int(byte_list[7], 16) + int(byte_list[8], 16) / 256\n    pressure_counts.append(count)\n\npressure_counts = np.array(pressure_counts)\n\npressure = pressure_from_frequency(pressure_counts, temperature_counts1, coefs=pres_coefs)\nprint("Druck in dbar:", pressure)\n'

In [11]:
'''
pressure_counts = []

for byte_list in bytes_list_array:
    count = int(byte_list[6], 16) * 256 + int(byte_list[7], 16) + int(byte_list[8], 16) / 256
    pressure_counts.append(count)

pressure_counts = np.array(pressure_counts)
compensation_voltage = np.array(1)

pressures = convert_pressure(pressure_counts,compensation_voltage,coefs= pres_coefs)

print("Pressures:", pressures,"db")
'''

'\npressure_counts = []\n\nfor byte_list in bytes_list_array:\n    count = int(byte_list[6], 16) * 256 + int(byte_list[7], 16) + int(byte_list[8], 16) / 256\n    pressure_counts.append(count)\n\npressure_counts = np.array(pressure_counts)\ncompensation_voltage = np.array(1)\n\npressures = convert_pressure(pressure_counts,compensation_voltage,coefs= pres_coefs)\n\nprint("Pressures:", pressures,"db")\n'

In [12]:
conductivity_counts1 = []
conductivity_counts2 = []

for byte_list in bytes_list_array:
    count1 = int(byte_list[3], 16) * 256 + int(byte_list[4], 16) + int(byte_list[5], 16) / 256
    count2 = int(byte_list[12], 16) * 256 + int(byte_list[13], 16) + int(byte_list[14], 16) / 256
    conductivity_counts1.append(count1)
    conductivity_counts2.append(count2)

conductivity_counts1 = np.array(conductivity_counts1)
conductivity_counts2 = np.array(conductivity_counts2)

conductifitys1 = convert_conductivity(conductivity_counts1, temperatures1, pressure_test_array, coefs=cond1_coefs)
conductifitys2 = convert_conductivity(conductivity_counts2, temperatures2, pressure_test_array, coefs=cond2_coefs)

print("Conductifitys1:", conductifitys1)
print("Conductifitys2:", conductifitys2)

Conductifitys1: [15.3307893  15.33055485 15.33078936 15.33074239 15.3305549  15.33074246
 15.33069557 15.33083621 15.33074246 15.33069564]
Conductifitys2: [15.32989468 15.33007497 15.33002992 15.32989469 15.33007505 15.33002992
 15.33002993 15.3299848  15.33002991 15.32998493]


In [13]:
salinitys1 = gsw.SP_from_C(conductifitys1, temperatures1, pressure_test_array)
salinitys2 = gsw.SP_from_C(conductifitys2, temperatures2, pressure_test_array)

print("Salinity1:", salinitys1,"PSU")
print("Salinity2:", salinitys2,"PSU")

Salinity1: [15.55212221 15.55192789 15.55203225 15.5519937  15.55168197 15.55205751
 15.55208277 15.55217384 15.55198062 15.55206942] PSU
Salinity2: [15.5513864  15.55157168 15.55157426 15.55146467 15.55155642 15.55160035
 15.55165253 15.55161602 15.55152209 15.55145915] PSU


In [14]:
voltages0 = []
voltages1 = []

for hex_string in hexstrings:
    v0 = hex_to_voltage(hex_string[30:33])
    v1 = hex_to_voltage(hex_string[33:36])
    voltages0.append(v0)
    voltages1.append(v1)

voltages0 = np.array(voltages0)
voltages1 = np.array(voltages1)

oxygens1 = convert_sbe43_oxygen(voltages0,temperatures1,pressure_test_array,salinitys1,coefs= oxy1_coefs)
oxygens2 = convert_sbe43_oxygen(voltages1,temperatures2,pressure_test_array,salinitys2,coefs= oxy2_coefs)
print("Oxygens1:", oxygens1,"ml/l")
print("Oxygens2:", oxygens2,"ml/l")


potential_densitys1 = potential_density_from_t_s_p(temperatures1,salinitys1,pressure_test_array)
potential_densitys2 = potential_density_from_t_s_p(temperatures2,salinitys2,pressure_test_array)

oxygens3 = convert_oxygen_to_umol_per_kg(oxygens1, potential_densitys1) *1000
oxygens4 = convert_oxygen_to_umol_per_kg(oxygens2, potential_densitys2) *1000
print("Oxygens1:", oxygens3,"umol/kg")
print("Oxygens2:", oxygens4,"umol/kg")

Oxygens1: [8.12350182 8.12840815 8.12832951 8.12347245 8.12356307 8.12355611
 8.12359115 8.12830742 8.12833235 8.12845219] ml/l
Oxygens2: [8.12986258 8.12990316 8.12992827 8.12989616 8.12996323 8.12993947
 8.12444201 8.12991309 8.12438609 8.12996118] ml/l
Oxygens1: [358.3635196  358.58001181 358.57651809 358.36226265 358.36635012
 358.36593255 358.36746887 358.57550118 358.57665784 358.58191457] umol/kg
Oxygens2: [358.64431987 358.64605787 358.64716327 358.64577657 358.64871212
 358.64764884 358.40511542 358.64647918 358.40268922 358.64864886] umol/kg


In [15]:
time_elapsed = Time_elapsed(hexstrings)
print("Time_Elapsed:",time_elapsed,"s")

Time_Elapsed: [0.0, 0.041666666666666664, 0.08333333333333333, 0.125, 0.16666666666666666, 0.20833333333333331, 0.25, 0.29166666666666663, 0.3333333333333333, 0.375] s


In [16]:
deg = 54
minutes = 9.31
latitude = deg + minutes / 60
depths = depth_from_pressure(pressure_test_array,latitude)
print("Deaths: ",depths)

descent_rate = np.diff(depths) / np.diff(time_elapsed)

print("Sinkraten (m/s):", descent_rate)

Deaths:  [1.13872506 1.18629568 1.18629568 1.13872506 1.24179473 1.18629568
 1.18629568 1.13872506 1.18629568 1.23485735]
Sinkraten (m/s): [ 1.14169491  0.         -1.14169491  2.47367198 -1.33197706  0.
 -1.14169491  1.14169491  1.16547995]


In [17]:
hex_string = "0CD08410767B80458F0CCA6F1107AE7D58A3987C41F0DEF2FFFFBE0000BB29512A089DB7004D638E5C91C465"
voltage2 = hex_to_voltage(hex_string[36:39])
voltage3 = hex_to_voltage(hex_string[39:42])
voltage4 = hex_to_voltage(hex_string[42:45])
voltage5 = hex_to_voltage(hex_string[45:48])
voltage6 = hex_to_voltage(hex_string[48:51])
voltage7 = hex_to_voltage(hex_string[51:54])
print(voltage2, voltage3, voltage4, voltage5, voltage6, voltage7)

2.021978021978022 1.1697191697191696 0.29548229548229554 0.3284493284493284 0.0 0.07936507936507964


In [18]:
# name 11 = par: PAR/Irradiance, Biospherical/Licor
voltages2 = []

for hex_string in hexstrings:
    v2 = hex_to_voltage(hex_string[36:39])
    voltages2.append(v2)
    
voltages2 = np.array(voltages2)

par = convert_par(voltages2, coefs = par_coefs)
print("Par:", par, "µmol photons m−2s−1")

Par: [9.24020531 9.26647434 9.26647434 9.26647434 9.24020531 9.21401003
 9.16183989 9.13586462 9.10996228 9.08413265] µmol photons m−2s−1


In [19]:
voltages3 = []

for hex_string in hexstrings:
    v3 = hex_to_voltage(hex_string[36:39])
    voltages3.append(v3)
    
voltages3 = np.array(voltages3)

In [20]:
# name 9 = flECO-AFL: Fluorescence, WET Labs ECO-AFL/FL [mg/m^3]



In [21]:
# name 10 = turbWETntu0: Turbidity, WET Labs ECO [NTU]

In [22]:
# name 11 = par: PAR/Irradiance, Biospherical/Licor

In [23]:
# name 12 = spar: SPAR, Biospherical/Licor 

In [29]:
# print("Pressures:", pressures,"db\n")

print("Temperatures1:", temperatures1,"°C\n")
print("Temperatures2:", temperatures2,"°C\n")

print("Conductifitys1:", conductifitys1,"mS/cm\n")
print("Conductifitys2:", conductifitys2,"mS/cm\n")

print("Oxygens1:", oxygens1,"ml/l\n")
print("Oxygens2:", oxygens2,"ml/l\n")
print("Oxygens3:", oxygens3,"umol/kg\n")
print("Oxygens4:", oxygens4,"umol/kg\n")

print("Par:", par, "µmol photons m−2s−1\n")

print("Time_Elapsed:", time_elapsed,"s\n")

print("Salinity1:", salinitys1,"PSU\n")
print("Salinity2:", salinitys2,"PSU\n")

Temperatures1: [3.56156397 3.56140342 3.56172452 3.56172452 3.56188507 3.56156397
 3.56140342 3.56156397 3.56172452 3.56140342] °C

Temperatures2: [3.56104427 3.56104427 3.56093531 3.56088084 3.56104427 3.56088084
 3.56077188 3.56077188 3.56104427 3.56104427] °C

Conductifitys1: [15.3307893  15.33055485 15.33078936 15.33074239 15.3305549  15.33074246
 15.33069557 15.33083621 15.33074246 15.33069564] mS/cm

Conductifitys2: [15.32989468 15.33007497 15.33002992 15.32989469 15.33007505 15.33002992
 15.33002993 15.3299848  15.33002991 15.32998493] mS/cm

Oxygens1: [8.12350182 8.12840815 8.12832951 8.12347245 8.12356307 8.12355611
 8.12359115 8.12830742 8.12833235 8.12845219] ml/l

Oxygens2: [8.12986258 8.12990316 8.12992827 8.12989616 8.12996323 8.12993947
 8.12444201 8.12991309 8.12438609 8.12996118] ml/l

Oxygens3: [358.3635196  358.58001181 358.57651809 358.36226265 358.36635012
 358.36593255 358.36746887 358.57550118 358.57665784 358.58191457] umol/kg

Oxygens4: [358.64431987 358.646057

In [30]:
name 0 = Pressure: -42139.332086015915 db                        --> Falsch
name 1 = Temp_1: 3.561563969710619 °C                     --> Richtig
name 2 = Temp_2: 3.561044266345732 °C                     --> Richtig
name 3 = Conductivity_1: 15.330789300015969 mS/cm         --> Richtig
name 4 = Conductivity_2: 15.32989468343113 mS/cm          --> Richtig
name 5 = Oxygen1: [8.12350182] ml/l                       --> Richtig
name 6 = Oxygen2: [8.12986258] ml/l                       --> Richtig
name 7 = Oxygen1: [358.3635196] umol/kg                   --> Richtig
name 8 = Oxygen2: [358.64431987] umol/kg                  --> Richtig

name 9 = flECO-AFL: Fluorescence, WET Labs ECO-AFL/FL [mg/m^3]   --> Noch nicht
name 10 = turbWETntu0: Turbidity, WET Labs ECO [NTU]             --> Noch nicht
name 11 = par: PAR/Irradiance, Biospherical/Licor         --> Richtig
name 12 = spar: SPAR, Biospherical/Licor                         --> Noch nicht
name 13 = timeS: Time, Elapsed [seconds]                  --> Richtig
name 14 = dz/dtM: Descent Rate [m/s]                             --> Later

name 15 = Salinity1: 15.5521222127295 PSU                 --> Richtig        
name 16 = Salinity2: 15.551386401977785 PSU               --> Richtig



name 0 = Pressure: -42139.332086015915 db                        --> Falsch

name 9 = flECO-AFL: Fluorescence, WET Labs ECO-AFL/FL [mg/m^3]   --> Noch nicht
name 10 = turbWETntu0: Turbidity, WET Labs ECO [NTU]             --> Noch nicht
name 12 = spar: SPAR, Biospherical/Licor                         --> Noch nicht

name 14 = dz/dtM: Descent Rate [m/s]                             --> Later

name 17 = flag:  0.000e+00                                --> ???






SyntaxError: invalid character '°' (U+00B0) (2971561078.py, line 2)